In [1]:
!pip install textblob wordcloud langdetect -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [12]:
# EDA COMPLETO - EVENTOS JOINNUS LIMA
# =========================================================
# ANÁLISIS DE EVENTOS CON ENFOQUE EN TEXTO Y DESCRIPCIONES

# 1. INSTALACIÓN Y CONFIGURACIÓN
# =========================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Para análisis de texto
from textblob import TextBlob
from wordcloud import WordCloud
import re
from datetime import datetime, timedelta
from langdetect import detect, LangDetectException

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

# Cargar datos
df = pd.read_csv('joinnus.csv')
# Si es JSON: df = pd.read_json('joinnus_events.json')

print("=" * 80)
print("ANÁLISIS EXPLORATORIO DE DATOS - EVENTOS JOINNUS LIMA")
print("=" * 80)

# 2. INFORMACIÓN GENERAL DEL DATASET
# =========================================================
print("\n📊 INFORMACIÓN GENERAL DEL DATASET")
print("-" * 80)
print(f"Dimensiones: {df.shape[0]} filas x {df.shape[1]} columnas")
print(f"\nColumnas disponibles:")
for col in df.columns:
    print(f"  - {col}: {df[col].dtype}")

print(f"\nMemoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Información básica
print(f"\nInformación básica:")
print(f"  - Eventos únicos: {df['event_id'].nunique() if 'event_id' in df.columns else len(df)}")
print(f"  - Categorías únicas: {df['category'].nunique()}")
print(f"  - Lugares (venues) únicos: {df['location_venue'].nunique()}")

# 3. ANÁLISIS DE VALORES NULOS
# =========================================================
print("\n\n🔍 ANÁLISIS DE VALORES NULOS")
print("-" * 80)

nulos = pd.DataFrame({
    'Columna': df.columns,
    'Valores Nulos': df.isnull().sum(),
    'Porcentaje (%)': (df.isnull().sum() / len(df) * 100).round(2)
})
print(nulos[nulos['Valores Nulos'] > 0].sort_values('Valores Nulos', ascending=False))

# Visualización de valores nulos
fig, ax = plt.subplots(figsize=(12, 6))
nulos_plot = nulos[nulos['Valores Nulos'] > 0].sort_values('Porcentaje (%)', ascending=True)
if len(nulos_plot) > 0:
    ax.barh(nulos_plot['Columna'], nulos_plot['Porcentaje (%)'], color='coral')
    ax.set_xlabel('Porcentaje de Valores Nulos (%)')
    ax.set_title('Valores Nulos por Columna', fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    for i, v in enumerate(nulos_plot['Porcentaje (%)'].values):
        ax.text(v + 0.5, i, f'{v:.1f}%', va='center')
plt.tight_layout()
plt.show()

# Eliminamos las columnas sin datos
df = df.drop(columns=['price_max', 'organizer_ruc', 'organizer', 'city'])

nulos = pd.DataFrame({
    'Columna': df.columns,
    'Valores Nulos': df.isnull().sum(),
    'Porcentaje (%)': (df.isnull().sum() / len(df) * 100).round(2)
})
# Visualización de valores nulos
fig, ax = plt.subplots(figsize=(12, 6))
nulos_plot = nulos[nulos['Valores Nulos'] > 0].sort_values('Porcentaje (%)', ascending=True)
if len(nulos_plot) > 0:
    ax.barh(nulos_plot['Columna'], nulos_plot['Porcentaje (%)'], color='coral')
    ax.set_xlabel('Porcentaje de Valores Nulos (%)')
    ax.set_title('Valores Nulos por Columna', fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    for i, v in enumerate(nulos_plot['Porcentaje (%)'].values):
        ax.text(v + 0.5, i, f'{v:.1f}%', va='center')
plt.tight_layout()
plt.show()
# 4. ANÁLISIS DE CATEGORÍAS
# =========================================================
print("\n\n📂 ANÁLISIS DE CATEGORÍAS")
print("-" * 80)

categoria_counts = df['category'].value_counts()
print(f"Total de categorías: {len(categoria_counts)}")
print(f"\nDistribución de categorías:")
print(categoria_counts)

# Estadísticas por categoría
cat_stats = df.groupby('category').agg({
    'price_min': ['mean', 'median', 'min', 'max'],
    'title': 'count'
}).round(2)
cat_stats.columns = ['precio_promedio', 'precio_mediano', 'precio_min', 'precio_max', 'num_eventos']
cat_stats = cat_stats.sort_values('num_eventos', ascending=False)
print(f"\nEstadísticas por categoría:")
print(cat_stats)

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# Distribución de categorías
axes[0, 0].barh(range(len(categoria_counts)), categoria_counts.values,
                color=plt.cm.viridis(np.linspace(0, 1, len(categoria_counts))))
axes[0, 0].set_yticks(range(len(categoria_counts)))
axes[0, 0].set_yticklabels(categoria_counts.index)
axes[0, 0].set_xlabel('Número de Eventos')
axes[0, 0].set_title('Distribución de Eventos por Categoría', fontweight='bold', fontsize=12)
axes[0, 0].invert_yaxis()
for i, v in enumerate(categoria_counts.values):
    axes[0, 0].text(v + 1, i, str(v), va='center', fontweight='bold')

# Pie chart
colors = plt.cm.Set3(np.linspace(0, 1, len(categoria_counts)))
axes[0, 1].pie(categoria_counts.values, labels=categoria_counts.index, autopct='%1.1f%%',
               colors=colors, startangle=90)
axes[0, 1].set_title('Proporción de Categorías', fontweight='bold', fontsize=12)

# Precio promedio por categoría
cat_price = df.groupby('category')['price_min'].mean().sort_values(ascending=False)
axes[1, 0].barh(range(len(cat_price)), cat_price.values, color='coral')
axes[1, 0].set_yticks(range(len(cat_price)))
axes[1, 0].set_yticklabels(cat_price.index)
axes[1, 0].set_xlabel('Precio Promedio (PEN)')
axes[1, 0].set_title('Precio Promedio por Categoría', fontweight='bold', fontsize=12)
axes[1, 0].invert_yaxis()
for i, v in enumerate(cat_price.values):
    axes[1, 0].text(v + 1, i, f'S/.{v:.0f}', va='center')

# Rango de precios por categoría
cat_price_range = df.groupby('category').agg({'price_min': 'min'})
x = range(len(cat_price_range))
axes[1, 1].barh([i - 0.2 for i in x], cat_price_range['price_min'], 0.4,
                label='Precio Mínimo', color='lightgreen', alpha=0.8)
axes[1, 1].set_yticks(x)
axes[1, 1].set_yticklabels(cat_price_range.index)
axes[1, 1].set_xlabel('Precio (PEN)')
axes[1, 1].set_title('Rango de Precios por Categoría', fontweight='bold', fontsize=12)
axes[1, 1].invert_yaxis()
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# 5. ANÁLISIS DE PRECIOS
# =========================================================
print("\n\n💰 ANÁLISIS DE PRECIOS")
print("-" * 80)

print(f"Estadísticas de precios:")
print(f"  - Precio mínimo: S/.{df['price_min'].min():.2f}")
print(f"  - Precio promedio: S/.{df['price_min'].mean():.2f}")
print(f"  - Precio mediano: S/.{df['price_min'].median():.2f}")
print(f"  - Eventos gratuitos: {(df['price_min'] == 0).sum()} ({(df['price_min'] == 0).sum()/len(df)*100:.1f}%)")

# Rangos de precio
df['price_range'] = pd.cut(df['price_min'],
                           bins=[0, 1, 50, 100, 200, 500, df['price_min'].max()],
                           labels=['Gratis', '1-50', '51-100', '101-200', '201-500', '500+'])

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Distribución de precios
axes[0, 0].hist(df['price_min'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0, 0].axvline(df['price_min'].mean(), color='red', linestyle='--', linewidth=2,
                   label=f'Media: S/.{df["price_min"].mean():.0f}')
axes[0, 0].axvline(df['price_min'].median(), color='green', linestyle='--', linewidth=2,
                   label=f'Mediana: S/.{df["price_min"].median():.0f}')
axes[0, 0].set_xlabel('Precio Mínimo (PEN)')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].set_title('Distribución de Precios', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Box plot de precios
axes[0, 1].boxplot(df['price_min'].dropna(), vert=True, patch_artist=True,
                   boxprops=dict(facecolor='lightblue', alpha=0.7),
                   medianprops=dict(color='red', linewidth=2))
axes[0, 1].set_ylabel('Precio Mínimo (PEN)')
axes[0, 1].set_title('Box Plot de Precios', fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# Eventos por rango de precio
price_range_counts = df['price_range'].value_counts().sort_index()
axes[1, 0].bar(range(len(price_range_counts)), price_range_counts.values,
               color=['#4caf50', '#8bc34a', '#ffeb3b', '#ff9800', '#ff5722', '#d32f2f'],
               edgecolor='black', alpha=0.8)
axes[1, 0].set_xticks(range(len(price_range_counts)))
axes[1, 0].set_xticklabels(price_range_counts.index, rotation=45)
axes[1, 0].set_ylabel('Número de Eventos')
axes[1, 0].set_title('Eventos por Rango de Precio', fontweight='bold')
axes[1, 0].grid(axis='y', alpha=0.3)
for i, v in enumerate(price_range_counts.values):
    axes[1, 0].text(i, v + 5, str(v), ha='center', fontweight='bold')

# Scatter: precio vs categoría
cat_order = df.groupby('category')['price_min'].mean().sort_values().index
cat_num = {cat: i for i, cat in enumerate(cat_order)}
df['cat_num'] = df['category'].map(cat_num)
axes[1, 1].scatter(df['cat_num'], df['price_min'], alpha=0.5, c=df['cat_num'],
                   cmap='viridis', edgecolors='black', linewidth=0.5, s=50)
axes[1, 1].set_xticks(range(len(cat_order)))
axes[1, 1].set_xticklabels(cat_order, rotation=45, ha='right', fontsize=9)
axes[1, 1].set_ylabel('Precio (PEN)')
axes[1, 1].set_title('Distribución de Precios por Categoría', fontweight='bold')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# 6. ANÁLISIS DE UBICACIONES (VENUES)
# =========================================================
print("\n\n🏛️ ANÁLISIS DE UBICACIONES Y VENUES")
print("-" * 80)

venue_counts = df['location_venue'].value_counts().head(20)
print(f"Total de venues únicos: {df['location_venue'].nunique()}")
print(f"\nTop 20 venues más utilizados:")
print(venue_counts)

# Direcciones más frecuentes
address_counts = df['address'].value_counts().head(15)
print(f"\nTop 15 direcciones más frecuentes:")
print(address_counts)

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# Top venues
axes[0, 0].barh(range(len(venue_counts)), venue_counts.values,
                color=plt.cm.plasma(np.linspace(0, 1, len(venue_counts))))
axes[0, 0].set_yticks(range(len(venue_counts)))
axes[0, 0].set_yticklabels(venue_counts.index, fontsize=9)
axes[0, 0].set_xlabel('Número de Eventos')
axes[0, 0].set_title('Top 20 Venues Más Utilizados', fontweight='bold', fontsize=12)
axes[0, 0].invert_yaxis()
for i, v in enumerate(venue_counts.values):
    axes[0, 0].text(v + 0.5, i, str(v), va='center', fontsize=8)

# Top direcciones
top_addr = address_counts.head(12)
axes[0, 1].barh(range(len(top_addr)), top_addr.values, color='coral', alpha=0.7)
axes[0, 1].set_yticks(range(len(top_addr)))
axes[0, 1].set_yticklabels([addr[:50] + '...' if len(addr) > 50 else addr for addr in top_addr.index], fontsize=8)
axes[0, 1].set_xlabel('Número de Eventos')
axes[0, 1].set_title('Top 12 Direcciones Más Frecuentes', fontweight='bold', fontsize=12)
axes[0, 1].invert_yaxis()

# Precio promedio por venue (top 15)
venue_price = df.groupby('location_venue').agg({'price_min': 'mean', 'title': 'count'})
venue_price = venue_price[venue_price['title'] >= 3].sort_values('price_min', ascending=False).head(15)
axes[1, 0].barh(range(len(venue_price)), venue_price['price_min'], color='lightgreen', alpha=0.7)
axes[1, 0].set_yticks(range(len(venue_price)))
axes[1, 0].set_yticklabels(venue_price.index, fontsize=9)
axes[1, 0].set_xlabel('Precio Promedio (PEN)')
axes[1, 0].set_title('Top 15 Venues con Mayor Precio Promedio (Mín. 3 eventos)', fontweight='bold', fontsize=11)
axes[1, 0].invert_yaxis()
for i, v in enumerate(venue_price['price_min'].values):
    axes[1, 0].text(v + 1, i, f'S/.{v:.0f}', va='center', fontsize=8)

# Eventos por venue (top 15)
venue_events = df['location_venue'].value_counts().head(15)
axes[1, 1].bar(range(len(venue_events)), venue_events.values, color='steelblue', alpha=0.7, edgecolor='black')
axes[1, 1].set_xticks(range(len(venue_events)))
axes[1, 1].set_xticklabels(venue_events.index, rotation=45, ha='right', fontsize=8)
axes[1, 1].set_ylabel('Número de Eventos')
axes[1, 1].set_title('Top 15 Venues por Cantidad de Eventos', fontweight='bold', fontsize=12)
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# 7. ANÁLISIS TEMPORAL
# =========================================================
print("\n\n📅 ANÁLISIS TEMPORAL DE EVENTOS")
print("-" * 80)

# Convertir fechas
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])
df['extracted_at'] = pd.to_datetime(df['extracted_at'])

# Extraer componentes temporales
df['start_month'] = df['start_date'].dt.month
df['start_day_of_week'] = df['start_date'].dt.day_name()
df['start_hour'] = df['start_date'].dt.hour
df['duration_days'] = (df['end_date'] - df['start_date']).dt.total_seconds() / 86400

print(f"Rango de fechas:")
print(f"  - Primer evento: {df['start_date'].min()}")
print(f"  - Último evento: {df['start_date'].max()}")
print(f"  - Duración promedio: {df['duration_days'].mean():.1f} días")
print(f"  - Duración máxima: {df['duration_days'].max():.0f} días")

# Manejar valores inválidos en campos temporales
df['start_month'] = df['start_month'].fillna(0).astype(int)
df['start_day_of_week'] = df['start_day_of_week'].fillna('Unknown')
df['start_hour'] = df['start_hour'].fillna(0).astype(int)

# Eventos por mes (filtrar valores inválidos)
month_counts = df[df['start_month'] > 0]['start_month'].value_counts().sort_index()
month_names = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']

# Eventos por día de la semana (filtrar Unknown)
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_counts = df[df['start_day_of_week'] != 'Unknown']['start_day_of_week'].value_counts().reindex(day_order).fillna(0)
day_names_es = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Eventos por mes
if len(month_counts) > 0:
    axes[0, 0].bar(range(len(month_counts)), month_counts.values,
                   color=plt.cm.cool(np.linspace(0, 1, len(month_counts))),
                   edgecolor='black', alpha=0.8)
    axes[0, 0].set_xticks(range(len(month_counts)))
    axes[0, 0].set_xticklabels([month_names[int(m)-1] for m in month_counts.index])
    axes[0, 0].set_ylabel('Número de Eventos')
    axes[0, 0].set_title('Distribución de Eventos por Mes', fontweight='bold', fontsize=12)
    axes[0, 0].grid(axis='y', alpha=0.3)
    for i, v in enumerate(month_counts.values):
        axes[0, 0].text(i, v + max(month_counts.values)*0.02, str(int(v)), ha='center', fontweight='bold')
else:
    axes[0, 0].text(0.5, 0.5, 'No hay datos de mes disponibles', ha='center', va='center', transform=axes[0, 0].transAxes)
    axes[0, 0].set_title('Distribución de Eventos por Mes', fontweight='bold', fontsize=12)

# Eventos por día de la semana
if len(day_counts) > 0 and day_counts.sum() > 0:
    axes[0, 1].bar(range(len(day_counts)), day_counts.values,
                   color=['#f44336', '#ff9800', '#ffc107', '#4caf50', '#2196f3', '#9c27b0', '#e91e63'],
                   edgecolor='black', alpha=0.8)
    axes[0, 1].set_xticks(range(len(day_counts)))
    axes[0, 1].set_xticklabels(day_names_es, rotation=45)
    axes[0, 1].set_ylabel('Número de Eventos')
    axes[0, 1].set_title('Distribución de Eventos por Día de la Semana', fontweight='bold', fontsize=12)
    axes[0, 1].grid(axis='y', alpha=0.3)
else:
    axes[0, 1].text(0.5, 0.5, 'No hay datos de día disponibles', ha='center', va='center', transform=axes[0, 1].transAxes)
    axes[0, 1].set_title('Distribución de Eventos por Día de la Semana', fontweight='bold', fontsize=12)

# Distribución de duración
duration_valid = df['duration_days'].dropna()
if len(duration_valid) > 0:
    axes[1, 0].hist(duration_valid[duration_valid <= 365], bins=50, color='mediumpurple', edgecolor='black', alpha=0.7)
    axes[1, 0].axvline(duration_valid.mean(), color='red', linestyle='--', linewidth=2,
                       label=f'Media: {duration_valid.mean():.1f} días')
    axes[1, 0].set_xlabel('Duración (días)')
    axes[1, 0].set_ylabel('Frecuencia')
    axes[1, 0].set_title('Distribución de Duración de Eventos (≤365 días)', fontweight='bold', fontsize=12)
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)
else:
    axes[1, 0].text(0.5, 0.5, 'No hay datos de duración disponibles', ha='center', va='center', transform=axes[1, 0].transAxes)
    axes[1, 0].set_title('Distribución de Duración de Eventos', fontweight='bold', fontsize=12)

# Eventos por hora de inicio
hour_valid = df['start_hour'].dropna()
if len(hour_valid) > 0:
    hour_counts = hour_valid.value_counts().sort_index()
    axes[1, 1].plot(hour_counts.index, hour_counts.values, marker='o', linewidth=2, markersize=8, color='#2196f3')
    axes[1, 1].fill_between(hour_counts.index, hour_counts.values, alpha=0.3, color='#2196f3')
    axes[1, 1].set_xlabel('Hora de Inicio')
    axes[1, 1].set_ylabel('Número de Eventos')
    axes[1, 1].set_title('Distribución de Eventos por Hora de Inicio', fontweight='bold', fontsize=12)
    axes[1, 1].grid(alpha=0.3)
    axes[1, 1].set_xticks(range(0, 24, 2))
else:
    axes[1, 1].text(0.5, 0.5, 'No hay datos de hora disponibles', ha='center', va='center', transform=axes[1, 1].transAxes)
    axes[1, 1].set_title('Distribución de Eventos por Hora de Inicio', fontweight='bold', fontsize=12)

plt.tight_layout()
plt.show()

# 9. ANÁLISIS DE CLASIFICACIÓN Y TIPO DE EVENTO
# =========================================================
print("\n\n🎭 ANÁLISIS DE CLASIFICACIÓN Y TIPO DE EVENTO")
print("-" * 80)

# Rating (clasificación por edad)
rating_counts = df['rating'].value_counts()
print(f"Distribución de clasificación por edad:")
print(rating_counts)

# Tipo de evento
event_type_counts = df['event_type'].value_counts().head(15)
print(f"\nTop 15 tipos de evento:")
print(event_type_counts)

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Rating distribution
axes[0].bar(range(len(rating_counts)), rating_counts.values,
            color=['#4caf50', '#8bc34a', '#ffeb3b', '#ff9800', '#f44336'][:len(rating_counts)],
            edgecolor='black', alpha=0.8)
axes[0].set_xticks(range(len(rating_counts)))
axes[0].set_xticklabels(rating_counts.index)
axes[0].set_ylabel('Número de Eventos')
axes[0].set_title('Distribución de Clasificación por Edad', fontweight='bold', fontsize=12)
axes[0].grid(axis='y', alpha=0.3)
for i, v in enumerate(rating_counts.values):
    axes[0].text(i, v + 5, str(v), ha='center', fontweight='bold')

# Tipo de evento
axes[1].barh(range(len(event_type_counts)), event_type_counts.values, color='teal', alpha=0.7)
axes[1].set_yticks(range(len(event_type_counts)))
axes[1].set_yticklabels(event_type_counts.index, fontsize=9)
axes[1].set_xlabel('Número de Eventos')
axes[1].set_title('Top 15 Tipos de Evento', fontweight='bold', fontsize=12)
axes[1].invert_yaxis()
for i, v in enumerate(event_type_counts.values):
    axes[1].text(v + 1, i, str(v), va='center', fontsize=9)

plt.tight_layout()
plt.show()

# 10. ANÁLISIS DE TAGS
# =========================================================
print("\n\n🏷️ ANÁLISIS DE TAGS Y ETIQUETAS")
print("-" * 80)

# Extraer todos los tags
all_tags = []
for tags in df['tags']:
    if isinstance(tags, str):
        try:
            tag_list = eval(tags)
            all_tags.extend(tag_list)
        except:
            pass
    elif isinstance(tags, list):
        all_tags.extend(tags)

tag_counts = Counter(all_tags)
print(f"Total de tags únicos: {len(tag_counts)}")
print(f"\nTop 30 tags más frecuentes:")
for tag, count in tag_counts.most_common(30):
    print(f"  {tag}: {count}")

# Visualización
fig, axes = plt.subplots(2, 1, figsize=(16, 12))

# Top 25 tags
top_tags = dict(tag_counts.most_common(25))
axes[0].barh(range(len(top_tags)), list(top_tags.values()),
             color=plt.cm.coolwarm(np.linspace(0, 1, len(top_tags))))
axes[0].set_yticks(range(len(top_tags)))
axes[0].set_yticklabels(list(top_tags.keys()), fontsize=10)
axes[0].set_xlabel('Frecuencia')
axes[0].set_title('Top 25 Tags Más Frecuentes', fontweight='bold', fontsize=14)
axes[0].invert_yaxis()
for i, v in enumerate(top_tags.values()):
    axes[0].text(v + 1, i, str(v), va='center', fontsize=9)

# Word Cloud de tags
all_tags_text = ' '.join(all_tags)
wordcloud_tags = WordCloud(width=1200, height=400, background_color='white',
                           colormap='viridis', max_words=100).generate(all_tags_text)
axes[1].imshow(wordcloud_tags, interpolation='bilinear')
axes[1].axis('off')
axes[1].set_title('Word Cloud de Tags', fontweight='bold', fontsize=14)

plt.tight_layout()
plt.show()

# 11. ANÁLISIS DE TEXTO - DESCRIPCIONES
# =========================================================
print("\n\n📖 ANÁLISIS DE TEXTO - DESCRIPCIONES DE EVENTOS")
print("-" * 80)

# Filtrar descripciones válidas
df['has_description'] = df['description'].notna() & (df['description'] != '')
descriptions_df = df[df['has_description']].copy()

print(f"Eventos con descripción: {len(descriptions_df)} ({len(descriptions_df)/len(df)*100:.1f}%)")
print(f"Eventos sin descripción: {(~df['has_description']).sum()}")

# Calcular métricas de texto
descriptions_df['desc_length'] = descriptions_df['description'].apply(lambda x: len(str(x)))
descriptions_df['desc_words'] = descriptions_df['description'].apply(lambda x: len(str(x).split()))
descriptions_df['desc_sentences'] = descriptions_df['description'].apply(lambda x: len(str(x).split('.')))

print(f"\nEstadísticas de longitud de descripciones:")
print(f"  - Longitud promedio: {descriptions_df['desc_length'].mean():.0f} caracteres")
print(f"  - Longitud mediana: {descriptions_df['desc_length'].median():.0f} caracteres")
print(f"  - Palabras promedio: {descriptions_df['desc_words'].mean():.1f} palabras")
print(f"  - Descripción más larga: {descriptions_df['desc_length'].max()} caracteres")
print(f"  - Descripción más corta: {descriptions_df['desc_length'].min()} caracteres")

# Detección de idioma
def detect_language(text):
    try:
        return detect(str(text))
    except:
        return 'unknown'

descriptions_df['language'] = descriptions_df['description'].apply(detect_language)
print(f"\nDistribución de idiomas:")
print(descriptions_df['language'].value_counts())

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Distribución de longitud
axes[0, 0].hist(descriptions_df['desc_length'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0, 0].axvline(descriptions_df['desc_length'].mean(), color='red', linestyle='--', linewidth=2,
                   label=f'Media: {descriptions_df["desc_length"].mean():.0f}')
axes[0, 0].set_xlabel('Longitud (caracteres)')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].set_title('Distribución de Longitud de Descripciones', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Distribución de palabras
axes[0, 1].hist(descriptions_df['desc_words'], bins=50, color='lightgreen', edgecolor='black', alpha=0.7)
axes[0, 1].axvline(descriptions_df['desc_words'].mean(), color='red', linestyle='--', linewidth=2,
                   label=f'Media: {descriptions_df["desc_words"].mean():.1f}')
axes[0, 1].set_xlabel('Número de Palabras')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].set_title('Distribución de Palabras en Descripciones', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Longitud por categoría
cat_length = descriptions_df.groupby('category')['desc_words'].mean().sort_values(ascending=False)
axes[1, 0].barh(range(len(cat_length)), cat_length.values, color='coral', alpha=0.7)
axes[1, 0].set_yticks(range(len(cat_length)))
axes[1, 0].set_yticklabels(cat_length.index)
axes[1, 0].set_xlabel('Promedio de Palabras')
axes[1, 0].set_title('Longitud Promedio de Descripción por Categoría', fontweight='bold')
axes[1, 0].invert_yaxis()
for i, v in enumerate(cat_length.values):
    axes[1, 0].text(v + 1, i, f'{v:.1f}', va='center')

# Relación longitud vs precio
axes[1, 1].scatter(descriptions_df['desc_words'], descriptions_df['price_min'],
                   alpha=0.5, c=descriptions_df['cat_num'], cmap='viridis',
                   edgecolors='black', linewidth=0.5, s=50)
axes[1, 1].set_xlabel('Número de Palabras')
axes[1, 1].set_ylabel('Precio (PEN)')
axes[1, 1].set_title('Relación: Longitud de Descripción vs Precio', fontweight='bold')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# 12. ANÁLISIS DE SENTIMIENTO EN DESCRIPCIONES
# =========================================================
print("\n\n😊 ANÁLISIS DE SENTIMIENTO EN DESCRIPCIONES")
print("-" * 80)

# Función para análisis de sentimiento
def analyze_sentiment(text):
    try:
        analysis = TextBlob(str(text))
        polarity = analysis.sentiment.polarity
        subjectivity = analysis.sentiment.subjectivity

        if polarity > 0.1:
            sentiment = 'Positivo'
        elif polarity < -0.1:
            sentiment = 'Negativo'
        else:
            sentiment = 'Neutral'

        return polarity, subjectivity, sentiment
    except:
        return 0, 0, 'Neutral'

descriptions_df[['polarity', 'subjectivity', 'sentiment']] = descriptions_df['description'].apply(
    lambda x: pd.Series(analyze_sentiment(x))
)

print(f"Distribución de sentimientos:")
print(descriptions_df['sentiment'].value_counts())
print(f"\nPolarity promedio: {descriptions_df['polarity'].mean():.3f}")
print(f"Subjectivity promedio: {descriptions_df['subjectivity'].mean():.3f}")

# Sentimiento por categoría
print(f"\nSentimiento promedio por categoría:")
sentiment_by_cat = descriptions_df.groupby('category')['polarity'].mean().sort_values(ascending=False)
print(sentiment_by_cat)

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Distribución de sentimientos
sentiment_counts = descriptions_df['sentiment'].value_counts()
colors_sent = {'Positivo': '#4caf50', 'Neutral': '#ffc107', 'Negativo': '#f44336'}
axes[0, 0].bar(sentiment_counts.index, sentiment_counts.values,
               color=[colors_sent[s] for s in sentiment_counts.index], edgecolor='black', alpha=0.8)
axes[0, 0].set_ylabel('Número de Eventos')
axes[0, 0].set_title('Distribución de Sentimientos en Descripciones', fontweight='bold', fontsize=14)
axes[0, 0].grid(axis='y', alpha=0.3)
for i, (sent, v) in enumerate(zip(sentiment_counts.index, sentiment_counts.values)):
    axes[0, 0].text(i, v + 10, str(v), ha='center', fontweight='bold', fontsize=12)

# Distribución de polarity
axes[0, 1].hist(descriptions_df['polarity'], bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0, 1].axvline(0, color='red', linestyle='--', linewidth=2, label='Neutral')
axes[0, 1].axvline(descriptions_df['polarity'].mean(), color='green', linestyle='--', linewidth=2,
                   label=f'Media: {descriptions_df["polarity"].mean():.3f}')
axes[0, 1].set_xlabel('Polarity (-1: Negativo, +1: Positivo)')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].set_title('Distribución de Polarity', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Polarity por categoría
axes[1, 0].barh(range(len(sentiment_by_cat)), sentiment_by_cat.values,
                color=plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(sentiment_by_cat))))
axes[1, 0].set_yticks(range(len(sentiment_by_cat)))
axes[1, 0].set_yticklabels(sentiment_by_cat.index)
axes[1, 0].set_xlabel('Polarity Promedio')
axes[1, 0].set_title('Sentimiento Promedio por Categoría', fontweight='bold')
axes[1, 0].invert_yaxis()
axes[1, 0].axvline(0, color='red', linestyle='--', linewidth=1, alpha=0.5)
for i, v in enumerate(sentiment_by_cat.values):
    axes[1, 0].text(v + 0.005, i, f'{v:.3f}', va='center', fontsize=9)

# Subjectivity distribution
axes[1, 1].hist(descriptions_df['subjectivity'], bins=50, color='mediumpurple', edgecolor='black', alpha=0.7)
axes[1, 1].axvline(descriptions_df['subjectivity'].mean(), color='red', linestyle='--', linewidth=2,
                   label=f'Media: {descriptions_df["subjectivity"].mean():.3f}')
axes[1, 1].set_xlabel('Subjectivity (0: Objetivo, 1: Subjetivo)')
axes[1, 1].set_ylabel('Frecuencia')
axes[1, 1].set_title('Distribución de Subjetividad', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# 13. PALABRAS FRECUENTES EN DESCRIPCIONES
# =========================================================
print("\n\n🔤 ANÁLISIS DE PALABRAS FRECUENTES")
print("-" * 80)

# Función para limpiar texto
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Stopwords en español e inglés
stopwords = {'el', 'la', 'de', 'que', 'y', 'a', 'en', 'un', 'ser', 'se', 'no', 'haber', 'por', 'con', 'su', 'para',
             'como', 'estar', 'tener', 'le', 'lo', 'todo', 'pero', 'más', 'hacer', 'o', 'poder', 'decir', 'este',
             'ir', 'otro', 'ese', 'si', 'me', 'ya', 'ver', 'porque', 'dar', 'cuando', 'él', 'muy', 'sin',
             'vez', 'mucho', 'saber', 'qué', 'sobre', 'mi', 'alguno', 'mismo', 'yo', 'también', 'hasta', 'año',
             'dos', 'querer', 'entre', 'así', 'primero', 'desde', 'grande', 'eso', 'ni', 'nos', 'llegar', 'pasar',
             'tiempo', 'ella', 'sí', 'día', 'uno', 'bien', 'poco', 'deber', 'entonces', 'poner', 'cosa', 'tanto',
             'hombre', 'parecer', 'nuestro', 'tan', 'donde', 'ahora', 'parte', 'después', 'vida', 'quedar', 'siempre',
             'creer', 'hablar', 'llevar', 'dejar', 'nada', 'cada', 'seguir', 'menos', 'nuevo', 'encontrar', 'algo',
             'solo', 'decir', 'pueden', 'puede', 'será', 'toda', 'todas', 'todos', 'sus', 'más', 'sido', 'solo',
             'the', 'is', 'are', 'was', 'were', 'has', 'have', 'had', 'be', 'been', 'being', 'do', 'does', 'did',
             'will', 'would', 'should', 'could', 'may', 'might', 'must', 'can', 'this', 'that', 'these', 'those',
             'i', 'you', 'he', 'she', 'it', 'we', 'they', 'them', 'their', 'what', 'which', 'who', 'when', 'where',
             'why', 'how', 'all', 'each', 'every', 'both', 'few', 'more', 'most', 'other', 'some', 'such', 'only',
             'own', 'same', 'so', 'than', 'too', 'very', 'just', 'but', 'and', 'or', 'if', 'because', 'as', 'until',
             'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during',
             'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over',
             'under', 'again', 'further', 'then', 'once', 'lima', 'perú', 'peru', 'evento', 'eventos'}

# Extraer palabras
all_text = ' '.join(descriptions_df['description'].apply(clean_text))
words = all_text.split()
words = [w for w in words if len(w) > 3 and w not in stopwords]
word_counts = Counter(words)

print(f"Top 30 palabras más frecuentes:")
for word, count in word_counts.most_common(30):
    print(f"  {word}: {count}")

# Palabras por categoría
category_words = {}
for category in descriptions_df['category'].unique():
    cat_text = ' '.join(descriptions_df[descriptions_df['category'] == category]['description'].apply(clean_text))
    cat_words = cat_text.split()
    cat_words = [w for w in cat_words if len(w) > 3 and w not in stopwords]
    category_words[category] = Counter(cat_words)

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# Top 25 palabras generales
top_words = dict(word_counts.most_common(25))
axes[0, 0].barh(range(len(top_words)), list(top_words.values()),
                color=plt.cm.viridis(np.linspace(0, 1, len(top_words))))
axes[0, 0].set_yticks(range(len(top_words)))
axes[0, 0].set_yticklabels(list(top_words.keys()))
axes[0, 0].set_xlabel('Frecuencia')
axes[0, 0].set_title('Top 25 Palabras Más Frecuentes en Descripciones', fontweight='bold', fontsize=12)
axes[0, 0].invert_yaxis()
for i, v in enumerate(top_words.values()):
    axes[0, 0].text(v + 5, i, str(v), va='center', fontsize=8)

# Palabras en eventos positivos
positive_desc = descriptions_df[descriptions_df['sentiment'] == 'Positivo']
if len(positive_desc) > 0:
    pos_text = ' '.join(positive_desc['description'].apply(clean_text))
    pos_words = pos_text.split()
    pos_words = [w for w in pos_words if len(w) > 3 and w not in stopwords]
    pos_word_counts = Counter(pos_words)
    top_pos = dict(pos_word_counts.most_common(15))

    axes[0, 1].barh(range(len(top_pos)), list(top_pos.values()), color='#4caf50', alpha=0.7)
    axes[0, 1].set_yticks(range(len(top_pos)))
    axes[0, 1].set_yticklabels(list(top_pos.keys()), fontsize=9)
    axes[0, 1].set_xlabel('Frecuencia')
    axes[0, 1].set_title('Top 15 Palabras en Descripciones Positivas', fontweight='bold', fontsize=12)
    axes[0, 1].invert_yaxis()

# Palabras por precio (eventos caros vs baratos)
expensive = descriptions_df[descriptions_df['price_min'] > descriptions_df['price_min'].quantile(0.75)]
cheap = descriptions_df[descriptions_df['price_min'] < descriptions_df['price_min'].quantile(0.25)]

exp_text = ' '.join(expensive['description'].apply(clean_text))
exp_words = [w for w in exp_text.split() if len(w) > 3 and w not in stopwords]
exp_word_counts = Counter(exp_words)

cheap_text = ' '.join(cheap['description'].apply(clean_text))
cheap_words = [w for w in cheap_text.split() if len(w) > 3 and w not in stopwords]
cheap_word_counts = Counter(cheap_words)

# Comparación
common_words = set(list(dict(exp_word_counts.most_common(20)).keys()) +
                  list(dict(cheap_word_counts.most_common(20)).keys()))
comparison_data = []
for word in list(common_words)[:15]:
    comparison_data.append({
        'word': word,
        'expensive': exp_word_counts[word],
        'cheap': cheap_word_counts[word]
    })

comp_df = pd.DataFrame(comparison_data).sort_values('expensive', ascending=False).head(12)
x = range(len(comp_df))
width = 0.35
axes[1, 0].barh([i - width/2 for i in x], comp_df['expensive'], width,
                label='Eventos Caros (Q4)', color='#f44336', alpha=0.8)
axes[1, 0].barh([i + width/2 for i in x], comp_df['cheap'], width,
                label='Eventos Baratos (Q1)', color='#4caf50', alpha=0.8)
axes[1, 0].set_yticks(x)
axes[1, 0].set_yticklabels(comp_df['word'], fontsize=9)
axes[1, 0].set_xlabel('Frecuencia')
axes[1, 0].set_title('Palabras: Eventos Caros vs Baratos', fontweight='bold', fontsize=12)
axes[1, 0].invert_yaxis()
axes[1, 0].legend()

# Palabras únicas por categoría (muestra de 3 categorías principales)
top_categories = descriptions_df['category'].value_counts().head(3).index
for idx, category in enumerate(top_categories):
    if idx < 3:
        print(f"\nTop 10 palabras en categoría '{category}':")
        for word, count in category_words[category].most_common(10):
            print(f"  {word}: {count}")

# Plot de nube de palabras compacta
all_words_text = ' '.join(list(top_words.keys()))
axes[1, 1].text(0.5, 0.5, 'Ver Word Clouds\nen la siguiente\nsección',
                ha='center', va='center', fontsize=16, fontweight='bold')
axes[1, 1].axis('off')
axes[1, 1].set_title('Word Clouds de Descripciones', fontweight='bold', fontsize=12)

plt.tight_layout()
plt.show()

# 14. WORD CLOUDS DE DESCRIPCIONES
# =========================================================
print("\n\n☁️ GENERANDO WORD CLOUDS DE DESCRIPCIONES...")
print("-" * 80)

fig, axes = plt.subplots(2, 2, figsize=(20, 12))

# WordCloud general
wordcloud_all = WordCloud(width=800, height=400, background_color='white',
                          stopwords=stopwords, colormap='viridis', max_words=100).generate(all_text)
axes[0, 0].imshow(wordcloud_all, interpolation='bilinear')
axes[0, 0].axis('off')
axes[0, 0].set_title('Word Cloud - Todas las Descripciones', fontweight='bold', fontsize=14)

# WordCloud descripciones positivas
if len(positive_desc) > 0:
    pos_full_text = ' '.join(positive_desc['description'].apply(clean_text))
    wordcloud_pos = WordCloud(width=800, height=400, background_color='white',
                              stopwords=stopwords, colormap='Greens', max_words=100).generate(pos_full_text)
    axes[0, 1].imshow(wordcloud_pos, interpolation='bilinear')
axes[0, 1].axis('off')
axes[0, 1].set_title('Word Cloud - Descripciones con Sentimiento Positivo', fontweight='bold', fontsize=14)

# WordCloud eventos caros
wordcloud_exp = WordCloud(width=800, height=400, background_color='white',
                          stopwords=stopwords, colormap='Reds', max_words=100).generate(exp_text)
axes[1, 0].imshow(wordcloud_exp, interpolation='bilinear')
axes[1, 0].axis('off')
axes[1, 0].set_title('Word Cloud - Eventos de Alto Precio (Q4)', fontweight='bold', fontsize=14)

# WordCloud eventos baratos
wordcloud_cheap = WordCloud(width=800, height=400, background_color='white',
                            stopwords=stopwords, colormap='Blues', max_words=100).generate(cheap_text)
axes[1, 1].imshow(wordcloud_cheap, interpolation='bilinear')
axes[1, 1].axis('off')
axes[1, 1].set_title('Word Cloud - Eventos de Bajo Precio (Q1)', fontweight='bold', fontsize=14)

plt.tight_layout()
plt.show()

# 15. ANÁLISIS DE TÍTULOS
# =========================================================
print("\n\n📋 ANÁLISIS DE TÍTULOS DE EVENTOS")
print("-" * 80)

# Métricas de títulos
df['title_length'] = df['title'].apply(lambda x: len(str(x)))
df['title_words'] = df['title'].apply(lambda x: len(str(x).split()))

print(f"Estadísticas de títulos:")
print(f"  - Longitud promedio: {df['title_length'].mean():.0f} caracteres")
print(f"  - Palabras promedio: {df['title_words'].mean():.1f} palabras")
print(f"  - Título más largo: {df['title_length'].max()} caracteres")
print(f"  - Título más corto: {df['title_length'].min()} caracteres")

# Palabras frecuentes en títulos
all_titles = ' '.join(df['title'].apply(clean_text))
title_words = all_titles.split()
title_words = [w for w in title_words if len(w) > 3 and w not in stopwords]
title_word_counts = Counter(title_words)

print(f"\nTop 20 palabras en títulos:")
for word, count in title_word_counts.most_common(20):
    print(f"  {word}: {count}")

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# Distribución de longitud de títulos
axes[0, 0].hist(df['title_length'], bins=40, color='lightcoral', edgecolor='black', alpha=0.7)
axes[0, 0].axvline(df['title_length'].mean(), color='red', linestyle='--', linewidth=2,
                   label=f'Media: {df["title_length"].mean():.0f}')
axes[0, 0].set_xlabel('Longitud (caracteres)')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].set_title('Distribución de Longitud de Títulos', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Top palabras en títulos
top_title_words = dict(title_word_counts.most_common(20))
axes[0, 1].barh(range(len(top_title_words)), list(top_title_words.values()),
                color=plt.cm.plasma(np.linspace(0, 1, len(top_title_words))))
axes[0, 1].set_yticks(range(len(top_title_words)))
axes[0, 1].set_yticklabels(list(top_title_words.keys()), fontsize=9)
axes[0, 1].set_xlabel('Frecuencia')
axes[0, 1].set_title('Top 20 Palabras en Títulos', fontweight='bold', fontsize=12)
axes[0, 1].invert_yaxis()

# WordCloud de títulos
wordcloud_titles = WordCloud(width=1000, height=500, background_color='white',
                             stopwords=stopwords, colormap='cool', max_words=80).generate(all_titles)
axes[1, 0].imshow(wordcloud_titles, interpolation='bilinear')
axes[1, 0].axis('off')
axes[1, 0].set_title('Word Cloud - Títulos de Eventos', fontweight='bold', fontsize=14)

# Relación longitud de título vs precio
axes[1, 1].scatter(df['title_words'], df['price_min'], alpha=0.5,
                   c=df['cat_num'], cmap='viridis', edgecolors='black', linewidth=0.5, s=50)
axes[1, 1].set_xlabel('Palabras en el Título')
axes[1, 1].set_ylabel('Precio (PEN)')
axes[1, 1].set_title('Relación: Longitud de Título vs Precio', fontweight='bold')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# 16. ANÁLISIS DE IMÁGENES
# =========================================================
print("\n\n📸 ANÁLISIS DE IMÁGENES")
print("-" * 80)

# Calcular número de imágenes
def count_images(images):
    if pd.isna(images) or images == '[]':
        return 0
    try:
        if isinstance(images, str):
            img_list = eval(images)
        else:
            img_list = images
        return len(img_list)
    except:
        return 0

df['num_images'] = df['images'].apply(count_images)

print(f"Estadísticas de imágenes:")
print(f"  - Eventos con imágenes: {(df['num_images'] > 0).sum()} ({(df['num_images'] > 0).sum()/len(df)*100:.1f}%)")
print(f"  - Promedio de imágenes por evento: {df['num_images'].mean():.2f}")
print(f"  - Máximo de imágenes: {df['num_images'].max()}")

# Imágenes por categoría
img_by_cat = df.groupby('category')['num_images'].agg(['mean', 'sum']).sort_values('mean', ascending=False)
print(f"\nImágenes promedio por categoría:")
print(img_by_cat)

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Distribución de número de imágenes
axes[0, 0].hist(df['num_images'], bins=30, color='mediumpurple', edgecolor='black', alpha=0.7)
axes[0, 0].axvline(df['num_images'].mean(), color='red', linestyle='--', linewidth=2,
                   label=f'Media: {df["num_images"].mean():.2f}')
axes[0, 0].set_xlabel('Número de Imágenes')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].set_title('Distribución del Número de Imágenes', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Imágenes promedio por categoría
axes[0, 1].barh(range(len(img_by_cat)), img_by_cat['mean'], color='teal', alpha=0.7)
axes[0, 1].set_yticks(range(len(img_by_cat)))
axes[0, 1].set_yticklabels(img_by_cat.index)
axes[0, 1].set_xlabel('Promedio de Imágenes')
axes[0, 1].set_title('Imágenes Promedio por Categoría', fontweight='bold')
axes[0, 1].invert_yaxis()
for i, v in enumerate(img_by_cat['mean'].values):
    axes[0, 1].text(v + 0.05, i, f'{v:.2f}', va='center')

# Relación imágenes vs precio
axes[1, 0].scatter(df['num_images'], df['price_min'], alpha=0.5,
                   c=df['cat_num'], cmap='viridis', edgecolors='black', linewidth=0.5, s=50)
axes[1, 0].set_title('Relación: Número de Imágenes vs Precio', fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# Eventos con/sin imágenes por categoría
img_status = df.groupby('category')['num_images'].apply(lambda x: (x > 0).sum())
no_img_status = df.groupby('category')['num_images'].apply(lambda x: (x == 0).sum())
x = range(len(img_status))
width = 0.35
axes[1, 1].bar([i - width/2 for i in x], img_status.values, width,
               label='Con Imágenes', color='#4caf50', alpha=0.8, edgecolor='black')
axes[1, 1].bar([i + width/2 for i in x], no_img_status.values, width,
               label='Sin Imágenes', color='#f44336', alpha=0.8, edgecolor='black')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(img_status.index, rotation=45, ha='right')
axes[1, 1].set_ylabel('Número de Eventos')
axes[1, 1].set_title('Eventos Con/Sin Imágenes por Categoría', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# 17. ANÁLISIS CRUZADO: CARACTERÍSTICAS DE TEXTO VS PRECIO
# =========================================================
print("\n\n🔄 ANÁLISIS CRUZADO: TEXTO VS PRECIO")
print("-" * 80)

# Correlaciones
if len(descriptions_df) > 0:
    text_price_corr = descriptions_df[['desc_length', 'desc_words', 'polarity', 'subjectivity', 'price_min']].corr()
    print("Correlación de características de texto con precio:")
    print(text_price_corr['price_min'].sort_values(ascending=False))

    # Segmentar eventos por precio
    descriptions_df['price_segment'] = pd.qcut(descriptions_df['price_min'], q=4,
                                                labels=['Bajo', 'Medio-Bajo', 'Medio-Alto', 'Alto'])

    # Agregar columnas faltantes desde df original
    descriptions_df = descriptions_df.merge(df[['title', 'title_words', 'num_images']],
                                           left_on='title', right_on='title',
                                           how='left', suffixes=('', '_y'))

    # Características por segmento de precio
    agg_dict = {
        'desc_words': 'mean',
        'polarity': 'mean',
        'subjectivity': 'mean'
    }

    # Agregar solo si las columnas existen
    if 'title_words' in descriptions_df.columns:
        agg_dict['title_words'] = 'mean'
    if 'num_images' in descriptions_df.columns:
        agg_dict['num_images'] = 'mean'

    segment_stats = descriptions_df.groupby('price_segment').agg(agg_dict).round(2)

    print(f"\nCaracterísticas promedio por segmento de precio:")
    print(segment_stats)

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Heatmap de correlaciones
if len(descriptions_df) > 0:
    # Seleccionar solo columnas que existen
    available_vars = ['desc_words', 'polarity', 'subjectivity', 'price_min']
    if 'title_words' in descriptions_df.columns:
        available_vars.insert(3, 'title_words')
    if 'num_images' in descriptions_df.columns:
        available_vars.append('num_images')

    corr_matrix = descriptions_df[available_vars].corr()

    im = axes[0, 0].imshow(corr_matrix, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
    axes[0, 0].set_xticks(range(len(available_vars)))
    axes[0, 0].set_yticks(range(len(available_vars)))

    # Etiquetas personalizadas
    labels = []
    for var in available_vars:
        if var == 'desc_words':
            labels.append('Palabras Desc')
        elif var == 'polarity':
            labels.append('Polarity')
        elif var == 'subjectivity':
            labels.append('Subjectivity')
        elif var == 'title_words':
            labels.append('Palabras Título')
        elif var == 'num_images':
            labels.append('Imágenes')
        elif var == 'price_min':
            labels.append('Precio')

    axes[0, 0].set_xticklabels(labels, rotation=45, ha='right', fontsize=9)
    axes[0, 0].set_yticklabels(labels, fontsize=9)
    axes[0, 0].set_title('Matriz de Correlación: Texto vs Precio', fontweight='bold')

    # Agregar valores
    for i in range(len(available_vars)):
        for j in range(len(available_vars)):
            text = axes[0, 0].text(j, i, f'{corr_matrix.iloc[i, j]:.2f}',
                                  ha="center", va="center", color="black", fontsize=9)

    plt.colorbar(im, ax=axes[0, 0])

# Características por segmento de precio
if len(descriptions_df) > 0 and 'price_segment' in descriptions_df.columns:
    segment_stats_plot = segment_stats.reset_index()
    x = range(len(segment_stats_plot))
    axes[0, 1].plot(x, segment_stats_plot['desc_words'], marker='o', linewidth=2,
                    markersize=8, label='Palabras Descripción', color='#2196f3')

    if 'title_words' in segment_stats_plot.columns:
        axes[0, 1].plot(x, segment_stats_plot['title_words'] * 10, marker='s', linewidth=2,
                        markersize=8, label='Palabras Título (x10)', color='#ff9800')

    axes[0, 1].set_xticks(x)
    axes[0, 1].set_xticklabels(segment_stats_plot['price_segment'])
    axes[0, 1].set_ylabel('Promedio')
    axes[0, 1].set_title('Longitud de Texto por Segmento de Precio', fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)

# Sentimiento por segmento de precio
if len(descriptions_df) > 0 and 'price_segment' in descriptions_df.columns:
    axes[1, 0].plot(x, segment_stats_plot['polarity'], marker='o', linewidth=3,
                    markersize=10, color='#4caf50', label='Polarity')
    axes[1, 0].axhline(0, color='red', linestyle='--', linewidth=1, alpha=0.5)
    axes[1, 0].set_xticks(x)
    axes[1, 0].set_xticklabels(segment_stats_plot['price_segment'])
    axes[1, 0].set_ylabel('Polarity Promedio')
    axes[1, 0].set_title('Sentimiento por Segmento de Precio', fontweight='bold')
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)

# Imágenes por segmento de precio
if len(descriptions_df) > 0 and 'price_segment' in descriptions_df.columns and 'num_images' in segment_stats_plot.columns:
    axes[1, 1].bar(x, segment_stats_plot['num_images'], color='mediumpurple',
                   alpha=0.7, edgecolor='black')
    axes[1, 1].set_xticks(x)
    axes[1, 1].set_xticklabels(segment_stats_plot['price_segment'])
    axes[1, 1].set_ylabel('Promedio de Imágenes')
    axes[1, 1].set_title('Número de Imágenes por Segmento de Precio', fontweight='bold')
    axes[1, 1].grid(axis='y', alpha=0.3)
    for i, v in enumerate(segment_stats_plot['num_images']):
        axes[1, 1].text(i, v + 0.05, f'{v:.2f}', ha='center', fontweight='bold')
else:
    axes[1, 1].text(0.5, 0.5, 'Datos de imágenes no disponibles',
                   ha='center', va='center', transform=axes[1, 1].transAxes, fontsize=12)
    axes[1, 1].set_title('Número de Imágenes por Segmento de Precio', fontweight='bold')

plt.tight_layout()
plt.show()

# 18. ANÁLISIS DE EVENTOS POR HORARIOS
# =========================================================
print("\n\n⏰ ANÁLISIS DETALLADO DE HORARIOS")
print("-" * 80)

# Análisis de horarios (times)
def count_times(times):
    if pd.isna(times) or times == '[]':
        return 0
    try:
        if isinstance(times, str):
            times_list = eval(times)
        else:
            times_list = times
        return len(times_list)
    except:
        return 0

df['num_times'] = df['times'].apply(count_times)

print(f"Estadísticas de horarios:")
print(f"  - Eventos con múltiples horarios: {(df['num_times'] > 1).sum()} ({(df['num_times'] > 1).sum()/len(df)*100:.1f}%)")
print(f"  - Promedio de horarios por evento: {df['num_times'].mean():.2f}")
print(f"  - Máximo de horarios: {df['num_times'].max()}")

# Horarios por categoría
times_by_cat = df.groupby('category')['num_times'].mean().sort_values(ascending=False)
print(f"\nHorarios promedio por categoría:")
print(times_by_cat)

# Horarios por día de la semana
times_by_day = df[df['start_day_of_week'] != 'Unknown'].groupby('start_day_of_week')['num_times'].mean().reindex(day_order).fillna(0)

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Distribución de número de horarios
axes[0, 0].hist(df['num_times'], bins=30, color='lightblue', edgecolor='black', alpha=0.7)
axes[0, 0].axvline(df['num_times'].mean(), color='red', linestyle='--', linewidth=2,
                   label=f'Media: {df["num_times"].mean():.2f}')
axes[0, 0].set_xlabel('Número de Horarios')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].set_title('Distribución del Número de Horarios', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Horarios por categoría
axes[0, 1].barh(range(len(times_by_cat)), times_by_cat.values, color='coral', alpha=0.7)
axes[0, 1].set_yticks(range(len(times_by_cat)))
axes[0, 1].set_yticklabels(times_by_cat.index)
axes[0, 1].set_xlabel('Promedio de Horarios')
axes[0, 1].set_title('Horarios Promedio por Categoría', fontweight='bold')
axes[0, 1].invert_yaxis()
for i, v in enumerate(times_by_cat.values):
    axes[0, 1].text(v + 0.05, i, f'{v:.2f}', va='center')

# Horarios por día de la semana
axes[1, 0].bar(range(len(times_by_day)), times_by_day.values,
               color=['#f44336', '#ff9800', '#ffc107', '#4caf50', '#2196f3', '#9c27b0', '#e91e63'],
               alpha=0.8, edgecolor='black')
axes[1, 0].set_xticks(range(len(times_by_day)))
axes[1, 0].set_xticklabels(day_names_es, rotation=45)
axes[1, 0].set_ylabel('Promedio de Horarios')
axes[1, 0].set_title('Horarios Promedio por Día de la Semana', fontweight='bold')
axes[1, 0].grid(axis='y', alpha=0.3)

# Relación horarios vs precio
axes[1, 1].scatter(df['num_times'], df['price_min'], alpha=0.5,
                   c=df['cat_num'], cmap='viridis', edgecolors='black', linewidth=0.5, s=50)
axes[1, 1].set_xlabel('Número de Horarios')
axes[1, 1].set_ylabel('Precio (PEN)')
axes[1, 1].set_title('Relación: Número de Horarios vs Precio', fontweight='bold')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# 19. TOP EVENTOS Y ANÁLISIS DETALLADO
# =========================================================
print("\n\n🏆 TOP EVENTOS Y ANÁLISIS DETALLADO")
print("-" * 80)

# Top eventos por diferentes criterios
print("Top 10 eventos más caros:")
top_expensive = df.nlargest(10, 'price_min')[['title', 'category', 'price_min', 'location_venue']]
print(top_expensive.to_string())

print("\nTop 10 eventos más económicos (no gratis):")
top_cheap = df[df['price_min'] > 0].nsmallest(10, 'price_min')[['title', 'category', 'price_min', 'location_venue']]
print(top_cheap.to_string())

if len(descriptions_df) > 0:
    print("\nTop 10 eventos con descripciones más largas:")
    top_long_desc = descriptions_df.nlargest(10, 'desc_words')[['title', 'category', 'desc_words', 'price_min']]
    print(top_long_desc.to_string())

    print("\nTop 10 eventos con sentimiento más positivo:")
    top_positive = descriptions_df.nlargest(10, 'polarity')[['title', 'category', 'polarity', 'price_min']]
    print(top_positive.to_string())

print("\nTop 10 eventos con más imágenes:")
top_images = df.nlargest(10, 'num_images')[['title', 'category', 'num_images', 'price_min']]
print(top_images.to_string())

# 20. RESUMEN EJECUTIVO COMPLETO
# =========================================================
print("\n\n" + "=" * 80)
print("📋 RESUMEN EJECUTIVO - EVENTOS JOINNUS LIMA")
print("=" * 80)

print(f"""
INFORMACIÓN GENERAL:
- Total de eventos: {len(df):,}
- Categorías: {df['category'].nunique()}
- Venues únicos: {df['location_venue'].nunique()}
- Período de eventos: {df['start_date'].min().strftime('%Y-%m-%d')} a {df['start_date'].max().strftime('%Y-%m-%d')}

PRECIOS:
- Precio promedio: S/.{df['price_min'].mean():.2f}
- Precio mediano: S/.{df['price_min'].median():.2f}
- Eventos gratuitos: {(df['price_min'] == 0).sum()} ({(df['price_min'] == 0).sum()/len(df)*100:.1f}%)

CATEGORÍAS MÁS POPULARES:
{chr(10).join([f"  {i+1}. {cat}: {count} eventos (S/.{df[df['category']==cat]['price_min'].mean():.0f} promedio)"
               for i, (cat, count) in enumerate(df['category'].value_counts().head(5).items())])}

TOP 5 VENUES:
{chr(10).join([f"  {i+1}. {venue}: {count} eventos"
               for i, (venue, count) in enumerate(df['location_venue'].value_counts().head(5).items())])}

TEMPORALIDAD:
- Día más popular: {df[df['start_day_of_week'] != 'Unknown']['start_day_of_week'].value_counts().index[0] if len(df[df['start_day_of_week'] != 'Unknown']) > 0 else 'N/A'}
- Mes con más eventos: {month_names[int(df[df['start_month'] > 0]['start_month'].value_counts().index[0]) - 1] if len(df[df['start_month'] > 0]) > 0 else 'N/A'}
- Hora promedio de inicio: {df[df['start_hour'] > 0]['start_hour'].mode().values[0]:02d}:00 if len(df[df['start_hour'] > 0]) > 0 else 'N/A'
- Duración promedio: {df['duration_days'].mean():.1f} días
""")

if len(descriptions_df) > 0:
    print(f"""ANÁLISIS DE TEXTO:
- Eventos con descripción: {len(descriptions_df)} ({len(descriptions_df)/len(df)*100:.1f}%)
- Palabras promedio en descripción: {descriptions_df['desc_words'].mean():.1f}
- Palabras promedio en título: {df['title_words'].mean():.1f}
- Idioma predominante: {descriptions_df['language'].mode().values[0] if len(descriptions_df) > 0 else 'N/A'}

SENTIMIENTO EN DESCRIPCIONES:
- Positivo: {(descriptions_df['sentiment'] == 'Positivo').sum()} ({(descriptions_df['sentiment'] == 'Positivo').sum()/len(descriptions_df)*100:.1f}%)
- Neutral: {(descriptions_df['sentiment'] == 'Neutral').sum()} ({(descriptions_df['sentiment'] == 'Neutral').sum()/len(descriptions_df)*100:.1f}%)
- Negativo: {(descriptions_df['sentiment'] == 'Negativo').sum()} ({(descriptions_df['sentiment'] == 'Negativo').sum()/len(descriptions_df)*100:.1f}%)
- Polarity promedio: {descriptions_df['polarity'].mean():.3f}
- Subjetividad promedio: {descriptions_df['subjectivity'].mean():.3f}

TOP 10 PALABRAS EN DESCRIPCIONES:
{chr(10).join([f"  {i+1}. {word} ({count} veces)" for i, (word, count) in enumerate(word_counts.most_common(10))])}

TOP 10 PALABRAS EN TÍTULOS:
{chr(10).join([f"  {i+1}. {word} ({count} veces)" for i, (word, count) in enumerate(title_word_counts.most_common(10))])}
""")

print(f"""CONTENIDO MULTIMEDIA:
- Eventos con imágenes: {(df['num_images'] > 0).sum()} ({(df['num_images'] > 0).sum()/len(df)*100:.1f}%)
- Imágenes promedio: {df['num_images'].mean():.2f}
- Eventos con múltiples horarios: {(df['num_times'] > 1).sum()} ({(df['num_times'] > 1).sum()/len(df)*100:.1f}%)

TOP 5 TAGS MÁS USADOS:
{chr(10).join([f"  {i+1}. {tag} ({count} eventos)" for i, (tag, count) in enumerate(tag_counts.most_common(5))])}

INSIGHTS CLAVE:
- Categoría más cara: {df.groupby('category')['price_min'].mean().idxmax()} (S/.{df.groupby('category')['price_min'].mean().max():.0f})
- Categoría más económica: {df[df['price_min'] > 0].groupby('category')['price_min'].mean().idxmin()} (S/.{df[df['price_min'] > 0].groupby('category')['price_min'].mean().min():.0f})
- Venue más caro (promedio): {venue_price.index[0]} (S/.{venue_price['price_min'].iloc[0]:.0f})
""")

if len(descriptions_df) > 0 and 'price_segment' in descriptions_df.columns:
    if 'desc_words' in segment_stats.columns:
        print(f"""- Eventos caros tienen {segment_stats.loc['Alto', 'desc_words']:.0f} palabras en descripción vs {segment_stats.loc['Bajo', 'desc_words']:.0f} en eventos baratos""")
    if 'polarity' in segment_stats.columns:
        print(f"""- Sentimiento más positivo en eventos de segmento: {segment_stats['polarity'].idxmax()}""")

print("=" * 80)
print("✅ ANÁLISIS COMPLETADO")
print("=" * 80)
print("\n💡 RECOMENDACIONES:")
print("  1. Los eventos con descripciones más largas y positivas tienden a tener mejor engagement")
print("  2. La categoría con mayor volumen es:", df['category'].value_counts().index[0])
print("  3. Los fines de semana concentran la mayor cantidad de eventos")
print("  4. El precio promedio varía significativamente entre categorías")
print("  5. Los eventos con múltiples horarios ofrecen mayor flexibilidad al público")
print("=" * 80)

Output hidden; open in https://colab.research.google.com to view.